In [2]:
PMID_input = input()

 C:\Users\svalb\OneDrive\Escritorio\


In [21]:
with open(PMID_input+"pmids_until_19991231.txt") as file_pmids_until_19991231:
    articles_pmids = [line.rstrip() for line in file_pmids_until_19991231]

with open(PMID_input+"pmids_20000101_until_20091231.txt") as file_pmids_20000101_until_20091231:
    articles_pmids_20000101_until_20091231 = [line.rstrip() for line in file_pmids_20000101_until_20091231]

with open(PMID_input+"pmids_20100101_until_20191231.txt") as file_pmids_20100101_until_20191231:
    articles_pmids_20100101_until_20191231 = [line.rstrip() for line in file_pmids_20100101_until_20191231]

with open(PMID_input+"pmids_20200101_until_20240701.txt") as file_pmids_20200101_until_20240701:
    articles_pmids_20200101_until_20240701 = [line.rstrip() for line in file_pmids_20200101_until_20240701]


articles_pmids.extend(articles_pmids_20000101_until_20091231)
articles_pmids.extend(articles_pmids_20100101_until_20191231)
articles_pmids.extend(articles_pmids_20200101_until_20240701)

In [23]:
len(articles_pmids)

37845132

In [ ]:
set_articles_pmids = set()
duplicated_articles_pmids = []

for pmid in articles_pmids:
    if pmid in set_articles_pmids:
        duplicated_articles_pmids.append(pmid)
    else:
        set_articles_pmids.add(pmid)